In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
%matplotlib inline

from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Dropout, Layer, Activation, Reshape, Permute
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
import random

from helper import *
from segnet_model import *

from distutils.version import LooseVersion

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


TensorFlow Version: 1.5.0


In [2]:
download_data()

In [3]:
segnet_basic = get_segnet_basic()

In [4]:
segnet_basic.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_1 (Layer)              (None, 320, 800, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 322, 802, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 320, 800, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 320, 800, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 320, 800, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 400, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 162, 402, 64)      0         
__________

In [5]:
#segnet_basic.summary()
segnet_basic.compile(loss="categorical_crossentropy", optimizer='adadelta', metrics=["accuracy"])

In [6]:
samples = getDataSetName()
shuffle(samples)

train_samples, validation_samples = train_test_split(samples, test_size=0.20)
validation_samples, test_samples = train_test_split(validation_samples, test_size=0.3)

In [7]:
#train_data, train_label = prep_data(test_samples)
#validate_data, validate_label = prep_data(validation_samples)
#test_data, test_label = prep_data(test_samples)
#epochs = 2
#batch_size = 10

#result = segnet_basic.fit(train_data, train_label, batch_size=batch_size, 
#                          epochs=epochs, verbose=1, shuffle=True)

In [8]:
train_generator = generator(train_samples, batch_size=4)
validation_generator = generator(validation_samples, batch_size=4)
test_generator = generator(test_samples, batch_size=4)

In [ ]:
segnet_basic.fit_generator(train_generator, steps_per_epoch= len(train_samples),
                    validation_data=validation_generator, validation_steps=len(validation_samples), epochs=2)

Epoch 1/2
1600/1600 [==============================] - 1638s 1s/step - loss: 0.1834 - acc: 0.9309 - val_loss: 0.0990 - val_acc: 0.9633
Epoch 2/2
1600/1600 [==============================] - 1567s 979ms/step - loss: 0.0679 - acc: 0.9747 - val_loss: 0.0722 - val_acc: 0.9734


In [ ]:
scores = segnet_basic.evaluate_generator(test_generator, len(test_samples))
print("Accuracy = ", scores)

In [ ]:
import h5py
from keras.models import model_from_json

segnet_basic.save('model_with_weight.h5')

segnet_basic.save_weights("model_weight.hdf5")
print("sava weight done..")    

json_string = segnet_basic.to_json()

with open("model.json", "w") as json_file:
    json_file.write(json_string)